<a href="https://colab.research.google.com/github/vedupandya/NLP/blob/main/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import datasets

In [6]:
dataset = datasets.load_dataset("cnn_dailymail",'1.0.0')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [7]:
data = dataset['train']['article'][:100]

In [8]:
import numpy as np
import spacy

In [9]:
nlp = spacy.load("en_core_web_sm")
stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [10]:
noun_phrases = []
for i in range(100):
    text = data[i]
    doc = nlp(text)
    for chunk in doc.noun_chunks:
        filtered_phrase = ' '.join(token.text for token in chunk if token.is_alpha and token.text.lower() not in stopwords)
        if filtered_phrase:
            noun_phrases.append(filtered_phrase.lower())

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
context_matrix = vectorizer.fit_transform(noun_phrases)

In [15]:
noun_phrases[10],noun_phrases[1]

('harry potter', 'england')

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
def cos_sim(np1, np2):
    np1_index = noun_phrases.index(np1)
    np2_index = noun_phrases.index(np2)
    cosine_sim = cosine_similarity(context_matrix[np1_index], context_matrix[np2_index])
    return cosine_sim[0][0]

In [25]:
noun_phrase_1 = noun_phrases[3]
noun_phrase_2 = noun_phrases[9]
similarity = cos_sim(noun_phrase_1, noun_phrase_2)
print(f"Cosine similarity between '{noun_phrase_1}' and '{noun_phrase_2}': {similarity}")

Cosine similarity between 'harry potter star daniel radcliffe' and 'daniel radcliffe': 0.6738574835244702


In [24]:
noun_phrase_1 = noun_phrases[9]
noun_phrase_2 = noun_phrases[10]
similarity = cos_sim(noun_phrase_1, noun_phrase_2)
print(f"Cosine similarity between '{noun_phrase_1}' and '{noun_phrase_2}': {similarity}")

Cosine similarity between 'daniel radcliffe' and 'harry potter': 0.0


In [23]:
noun_phrase_1 = noun_phrases[10]
noun_phrase_2 = noun_phrases[1]
similarity = cos_sim(noun_phrase_1, noun_phrase_2)
print(f"Cosine similarity between '{noun_phrase_1}' and '{noun_phrase_2}': {similarity}")

Cosine similarity between 'harry potter' and 'england': 0.0


As we can see, context matrix constructed from tf-idf does not capture relationships derived from extrinsic context very well while it does capture some of the relationships derived from intrinsic context